<a href="https://colab.research.google.com/github/bsrikanth24/Best-websites-a-programmer-should-visit/blob/master/Convert_a_string_column_in_a_pyspark_dataframe_to_List_of_Dictionaries%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
data = [
    {
    "employee_id": 873547690,
    "employee_name":"abc",
    "emp_deptname": "sales",
    "emp_endpoint": "abc@abc01storage",
    "emp_folder": "messages",
    "emp_address": [
                {
                    "type": "Home",
                    "city": "Delhi",
                    "apartment_number": "H.Number 124, B block, XYZ Towers",
                    "pincode": "123abc"
                },
                {
                    "type": "Residential",
                    "city": "Delhi",
                    "apartment_number": "Post Office 12A, Sector 22",
                    "pincode": "456xyz"
                }
            ],
    'event_timestamp': '1995-07-16 13:10:43'
        },
]

In [21]:
schema = "employee_id long, employee_name string, emp_deptname string, emp_endpoint string, emp_folder string, emp_address string, event_timestamp string"

In [22]:
schema_str = """
    employee_id long,
    employee_name string,
    emp_deptname string,
    emp_endpoint string,
    emp_folder string,
    emp_address array<struct<type string, city string, apartment_number string, pincode string>>,
    event_timestamp string
"""

In [12]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate()

In [23]:
data_sdf = spark.createDataFrame(data, schema_str)

data_sdf.show(truncate=False)

+-----------+-------------+------------+----------------+----------+--------------------------------------------------------------------------------------------------------------------+-------------------+
|employee_id|employee_name|emp_deptname|emp_endpoint    |emp_folder|emp_address                                                                                                         |event_timestamp    |
+-----------+-------------+------------+----------------+----------+--------------------------------------------------------------------------------------------------------------------+-------------------+
|873547690  |abc          |sales       |abc@abc01storage|messages  |[{Home, Delhi, H.Number 124, B block, XYZ Towers, 123abc}, {Residential, Delhi, Post Office 12A, Sector 22, 456xyz}]|1995-07-16 13:10:43|
+-----------+-------------+------------+----------------+----------+------------------------------------------------------------------------------------------------------------

In [24]:
data_sdf.printSchema()

root
 |-- employee_id: long (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- emp_deptname: string (nullable = true)
 |-- emp_endpoint: string (nullable = true)
 |-- emp_folder: string (nullable = true)
 |-- emp_address: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- apartment_number: string (nullable = true)
 |    |    |-- pincode: string (nullable = true)
 |-- event_timestamp: string (nullable = true)



In [18]:
data_sdf.select("emp_address").schema

StructType([StructField('emp_address', ArrayType(StructType([StructField('type', StringType(), True), StructField('city', StringType(), True), StructField('apartment_number', StringType(), True), StructField('pincode', StringType(), True)]), True), True)])

In [25]:
emp_address = data_sdf.select("emp_address").collect()
list(emp_address[0]["emp_address"])

[Row(type='Home', city='Delhi', apartment_number='H.Number 124, B block, XYZ Towers', pincode='123abc'),
 Row(type='Residential', city='Delhi', apartment_number='Post Office 12A, Sector 22', pincode='456xyz')]

In [ ]:
#Input DF
    # +-----+-------+
    # |group|subject|
    # +-----+-------+
    # |    A|   Math|
    # |    A|Physics|
    # |    B|Science|
    # +-----+-------+

In [31]:
from pyspark.sql import Row

data_input = [Row(group='A', subject='Math'),
              Row(group='A', subject='Physics'),
              Row(group='B', subject='Science')]

df1 = spark.createDataFrame(data_input)

df1.show()

+-----+-------+
|group|subject|
+-----+-------+
|    A|   Math|
|    A|Physics|
|    B|Science|
+-----+-------+



In [33]:
from pyspark.sql import functions as F
from pyspark.sql import Row

df1 = df1.groupBy("group").agg(F.collect_list("subject").alias("subject")).orderBy("group")
df1.show(truncate=False)

dict = {row['group']:row['subject'] for row in df1.collect()}
print(dict)

+-----+---------------+
|group|subject        |
+-----+---------------+
|A    |[Math, Physics]|
|B    |[Science]      |
+-----+---------------+

{'A': ['Math', 'Physics'], 'B': ['Science']}


In [37]:
from pyspark.sql import DataFrame
df= sc.parallelize([(1,2,3), (4,5,7)]).toDF(["a", "b", "c"])
if df is not None and isinstance(df,DataFrame):

      print("dataframe exists")

NameError: name 'sc' is not defined

Scenario 1:

Input data:

col1|col2|date
100|Austin|2021-01-10
100|Newyork|2021-02-15
100|Austin|2021-03-02
Expected output with CDC:

col1|col2|start_date|end_date
100|Austin|2021-01-10|2021-02-15
100|Newyork|2021-02-15|2021-03-02
100|Austin|2021-03-02|2099-12-31
In sequence there is a change in col2 values and want to maintain CDC

In [44]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession \
    .builder \
    .appName("SO") \
    .getOrCreate()

df = spark.createDataFrame(
    [(100, "Austin", "2021-01-10"),
(100, "Newyork", "2021-02-15"),
(100, "Austin", "2021-03-02"),
    ],
    ['col1', 'col2', 'date']
)

df.show()

df1 = df.withColumn("start_date", F.to_date("date"))

w = Window.partitionBy("col1",).orderBy("start_date")

df_1 = df1.withColumn("rn", F.row_number().over(w))

df_1.createOrReplaceTempView("temp")

df_dupe = spark.sql('select temp.col1,temp.col2,temp.start_date, case when temp.col1=temp_2.col1 and temp.col2=temp_2.col2 then "delete" else "no-delete" end as dupe  from temp left join temp as temp_2 '
                    'on temp.col1=temp_2.col1 and temp.col2=temp_2.col2 and temp.rn-1 = temp_2.rn order by temp.start_date  ')

df_dupe.filter(F.col("dupe")=="no-delete").drop("dupe")\
    .withColumn("end_date", F.coalesce(F.lead("start_date").over(w),F.lit("2099-12-31"))).show()

+----+-------+----------+
|col1|   col2|      date|
+----+-------+----------+
| 100| Austin|2021-01-10|
| 100|Newyork|2021-02-15|
| 100| Austin|2021-03-02|
+----+-------+----------+

+----+-------+----------+----------+
|col1|   col2|start_date|  end_date|
+----+-------+----------+----------+
| 100| Austin|2021-01-10|2021-02-15|
| 100|Newyork|2021-02-15|2021-03-02|
| 100| Austin|2021-03-02|2099-12-31|
+----+-------+----------+----------+



Scenario 2:

Input:

col1|col2|date
100|Austin|2021-01-10
100|Austin|2021-03-02  -> I want to eliminate this version because there is no change in col1 and col2 values between records.
Expected Output:

 col1|col2|start_date|end_date
 100|Austin|2021-01-10|2099-12-31

In [50]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("CDC_Example").getOrCreate()

data = spark.createDataFrame(
    [(100, "Austin", "2021-01-10"),
   (100, "Austin", "2021-03-02")],
    ['col1', 'col2', 'date']
)
df = data

# Convert date string to date type
df = df.withColumn("date", F.to_date("date"))

# Define window partitioned by col1 ordered by date
w = Window.partitionBy("col1").orderBy("date")

# Get previous col2 value
df = df.withColumn("prev_col2", F.lag("col2").over(w))

# Filter rows where col2 != prev_col2 or prev_col2 is null (first record)
df_filtered = df.filter((F.col("col2") != F.col("prev_col2")) | F.col("prev_col2").isNull())

# Add start_date and end_date columns
df_final = df_filtered.withColumnRenamed("date", "start_date") \
                      .withColumn("end_date", F.lit("2099-12-31")) \
                      .select("col1", "col2", "start_date", "end_date")

df_final.show()

+----+------+----------+----------+
|col1|  col2|start_date|  end_date|
+----+------+----------+----------+
| 100|Austin|2021-01-10|2099-12-31|
+----+------+----------+----------+



In [51]:
InputDF
    # +-------------+----------+
    # |UNIQUE_MEM_ID|      DATE|
    # +-------------+----------+
    # |         1156|      null|
    # |         3787|2016-07-05|
    # |         1156|      null|
    # +-------------+----------+

NameError: name 'InputDF' is not defined

In [53]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Ensure a SparkSession is available
spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate()


data_input_df = [Row(UNIQUE_MEM_ID=1156, DATE=None),
                 Row(UNIQUE_MEM_ID=3787, DATE='2016-07-05'),
                 Row(UNIQUE_MEM_ID=1156, DATE=None)]

InputDF = spark.createDataFrame(data_input_df)

InputDF.show()

+-------------+----------+
|UNIQUE_MEM_ID|      DATE|
+-------------+----------+
|         1156|      NULL|
|         3787|2016-07-05|
|         1156|      NULL|
+-------------+----------+



In [55]:
InputDF.select( *[ F.when(F.col(column).isNull(),'').otherwise(F.col(column)).alias(column) for column in InputDF.columns]).show()

+-------------+----------+
|UNIQUE_MEM_ID|      DATE|
+-------------+----------+
|         1156|          |
|         3787|2016-07-05|
|         1156|          |
+-------------+----------+

